In [50]:
##necessory imports
import json
import os
import shutil
import spacy  
import re
import string
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
import numpy as np
import warnings
warnings.filterwarnings("ignore")
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
import spacy
nlp =spacy.load('en_core_web_sm')

C:\Users\Hp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Traceback (most recent call last):
  File "C:\Users\Hp\anaconda3\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Hp\anaconda3\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\Hp\anaconda3\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exceptio

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [83]:
## class to get a list of stop words
def dataFromTextFile(filepath):
    stop_wordsList = []
    with open(filepath) as f:
        lines = f.read().splitlines()
        for line in lines:
            stop_wordsList.append(line)
        return ( stop_wordsList)

In [84]:
## class to preprocess training data
def data_preprocessing_train(data_dict,filepath):
    stop_words = dataFromTextFile(filepath)
    nlp =spacy.load('en_core_web_sm')
    pattern = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    df = pd.DataFrame(columns=['target', 'text'])
    for key in data_dict.keys():
        clean_text = []
        for line in data_dict[key]:
            # clean_data = []
            doc = nlp(line)
            for token in doc:
                clean = re.sub(pattern, '', str(token.lemma_).lower())
                if clean not in string.punctuation:
                    if clean not in stop_words:
                        clean_text.append(clean)
            # clean_text.append(clean_data)
        df = df.append({'target': key, 'text': clean_text}, ignore_index=True)
        print(df)
    return df

In [85]:
def preprocess_training_data(jsonFilePath, stop_words):
    with open(jsonFilePath, 'r') as f:
        data_dict = json.load(f)
       
    #### Data Cleaning
    clean_df = data_preprocessing_train(data_dict,stop_words)
    # converting preprocesed data from list to string to use in tfIdf
    clean_df['text'] = [" ".join(value) for value in clean_df['text'].values]

    return clean_df


In [86]:
class Train:

    def __init__(self, stopWordsFilePath):
        self.stop_words = stopWordsFilePath

    def training_model(self, jsonFilePath):
        data_df = preprocess_training_data(jsonFilePath, self.stop_words)

        TfidfVect = TfidfVectorizer()
        TfidfVect.fit(data_df['text'])

        # saving vector for prediciton
        with open("Desktop/TrainingDataModel/vectorizer.pickle", 'wb') as f:
                  pickle.dump(TfidfVect, f)
            

        #  train set and prediction set
        x = data_df['text']
        y = data_df['target']

        # splitting training and test data
        # x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state= 42)

        x_vector = TfidfVect.transform(x)
        # x_test_vector = TfidfVect.transform(x_test)

        # training data using SVM ##
        # model = svm.SVC(C=1.0, kernel='gaussian', degree=3, gamma='auto')
        model = naive_bayes.MultinomialNB()
        model.fit(x_vector, y)

        # y_pred = model.predict(x_test_vector)
        # score1 = metrics.accuracy_score(y_test, y_pred)

        # save the model to disk
        with open('modelForPrediction.sav', 'wb') as f:
            pickle.dump(model, f)

        return ("Success")


In [87]:
## class to preprocess the preddiction data
def data_preprocessing_predict(text_list,filepath):
    stop_words = dataFromTextFile(filepath)
    nlp = spacy.load('en_core_web_sm') # preprocessing library spacy
    pattern = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

    clean_text = []
    for data in text_list:
        clean_data = []
        doc = nlp(data)
        for token in doc:
            clean = re.sub(pattern, '', str(token.lemma_).lower())
            if clean not in string.punctuation:
                if clean not in stop_words:
                    clean_data.append(clean)
        clean_text.append(clean_data)
    return clean_text

In [88]:
class Predict:

    def __init__(self, stopWordsFilePath):
        self.nlp =spacy.load("en_core_web_sm")
        self.stop_words_path = stopWordsFilePath
        

    def executePreocessing(self, text, jsonFilePath,modelPath,vectorPath):
        df_pred = pd.DataFrame([text],columns=['text'])
        df_pred['text'] = data_preprocessing_predict(df_pred['text'], self.stop_words_path )
        df_pred['text'] = [" ".join(value) for value in df_pred['text'].values]

        with open(vectorPath,'rb') as f:
            vectorizer = pickle.load(f)
        with open(modelPath, 'rb') as f:
            model = pickle.load(f)
        pred_vector_ = vectorizer.transform(df_pred['text'])
        prediction = model.predict(pred_vector_)
        predictedProbability = model.predict_proba(pred_vector_)
        if list(predictedProbability.flatten())[0] == list(predictedProbability.flatten())[1]:
            return "UNKNOWN"
        elif list(predictedProbability.flatten())[np.argmax(predictedProbability)] > .5:
            return prediction
        else:
            return "UNKNOWN"


In [89]:
class Client:
    def __init__(self):
        stopWordsFilePath = "Desktop/data/stopwords.txt"
        self.predictObj = Predict(stopWordsFilePath)
        self.trainObj = Train(stopWordsFilePath)
        

In [90]:
clntApp =Client()

In [91]:
def trainModel():

    try:
        
        training_file_path="Desktop/trainingData.json"
    
        modelscore = clntApp.trainObj.training_model(training_file_path)
    
    except ValueError as val:
        print ("Value not found inside  json trainingData", val)
    except KeyError as keyval:
        print ("Key value error incorrect key passed", keyval)
    except Exception as e:
        print ((str(e)))

    print ("Success")

In [92]:
train=trainModel()

  target                                               text
0      0  [awww, bummer, shoulda, get, david, carr, thir...
  target                                               text
0      0  [awww, bummer, shoulda, get, david, carr, thir...
1      4  [followinq, shud, tha, add, tweetie, new, ipho...
Success


In [93]:
def predictRoute():
    try:
            text= "@switchfoot http:/,witpic.com/2y1zl - Awww, that's a bummer.  You shoulda got "
            jsonFilePath ="Desktop/trainingData.json"
            modelPath = "modelForPrediction.sav"
            vectorPath ="Desktop/TrainingDataModel/vectorizer.pickle"
            result = clntApp.predictObj.executePreocessing(text, jsonFilePath,modelPath,vectorPath)
            print(result)
    except ValueError:
        print ("Value not found inside  json trainingData")
    except KeyError:
        print ("Key value error incorrect key passed")
    except Exception as e:
        print ((str(e)))


In [94]:
predict=predictRoute()

['0']
